In [5]:
%reset
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch
import geomloss

import defmod as dm

torch.set_default_tensor_type(torch.FloatTensor)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Loading the datasets
source_raw = pickle.load(open('../data/basi2btemp.pkl', 'rb'))
target_raw = pickle.load(open('../data/basi2target.pkl', 'rb'))
data = pickle.load(open('../data/data_acropetal.pkl', 'rb'))

pos_source = torch.tensor(data['source_silent']).type(torch.get_default_dtype())
pos_implicit0 = torch.tensor(data['source_implicit0']).type(torch.get_default_dtype())
pos_implicit1 = torch.tensor(data['source_implicit1']).type(torch.get_default_dtype())
pos_target = torch.tensor(data['target_silent']).type(torch.get_default_dtype())

# Some rescaling for the source
Dx = 0.
Dy = 0.
height_source = 90.
height_target = 495.

smin, smax = torch.min(pos_source[:, 1]), torch.max(pos_source[:, 1])
sscale = height_source / (smax - smin)
pos_source[:, 0] = Dx + sscale * (pos_source[:, 0] - torch.mean(pos_source[:, 0]))
pos_source[:, 1] = Dy - sscale * (pos_source[:, 1] - smax)
pos_implicit0[:, 0] = Dx + sscale * (pos_implicit0[:, 0] - torch.mean(pos_implicit0[:, 0]))
pos_implicit0[:, 1] = Dy - sscale * (pos_implicit0[:, 1] - smax)
pos_implicit1[:, 0] = Dx + sscale * (pos_implicit1[:, 0] - torch.mean(pos_implicit1[:, 0]))
pos_implicit1[:, 1] = Dy - sscale * (pos_implicit1[:, 1] - smax)

# Some rescaling for the target
tmin, tmax = torch.min(pos_target[:, 1]), torch.max(pos_target[:, 1])
tscale = height_target / (tmax - tmin)
pos_target[:, 0] = tscale * (pos_target[:, 0] - torch.mean(pos_target[:, 0]))
pos_target[:, 1] = - tscale * (pos_target[:, 1] - tmax)

# Compute an AABB for plotting
aabb = dm.usefulfunctions.AABB.build_from_points(pos_target)
aabb.squared()

In [11]:
# Some plots
%matplotlib qt5

plt.subplot(1, 2, 1)
plt.axis(aabb.get_list())
plt.title('Source')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')

plt.subplot(1, 2, 2)
plt.axis(aabb.get_list())
plt.title('Target')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')

plt.show()

In [46]:
# Setting up the modules
# Local translation module
sigma0 = 10.
nu0 = 0.001
coeff0 = 100.
implicit0 = dm.implicitmodules.ImplicitModule0(dm.manifold.Landmarks(2, pos_implicit0.shape[0], gd=pos_implicit0.view(-1).requires_grad_()), sigma0, nu0, coeff0)

# Global translation module
sigma00 = 800.
nu00 = 0.001
coeff00 = 0.01
implicit00 = dm.implicitmodules.ImplicitModule0(dm.manifold.Landmarks(2, 1, gd=torch.tensor([0., 0.], requires_grad=True)), sigma00, nu00, coeff00)

# Elastic modules
sigma1 = 100.
nu1 = 0.001
coeff1 = 0.000001
C = torch.zeros(pos_implicit1.shape[0], 2, 1)
K, L = 10, height_source
a, b = 1./L, 3.
z = a*(pos_implicit1[:, 1] - Dy)
C[:, 1, 0] = K * ((1 - b) * z**2 + b * z)
C[:, 0, 0] = 0.8 * C[:, 1, 0]
th = 0. * math.pi * torch.ones(pos_implicit1.shape[0])
R = torch.stack([dm.usefulfunctions.rot2d(t) for t in th])

implicit1 = dm.implicitmodules.ImplicitModule1(dm.manifold.Stiefel(2, pos_implicit1.shape[0], gd=(pos_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_())), C, sigma1, nu1, coeff1)

In [47]:
# Setting up the model and start the fitting loop
model = dm.models.ModelCompoundWithPointsRegistration((pos_source, torch.ones(pos_source.shape[0])), [implicit0, implicit00, implicit1], [True, True, True])
costs = model.fit((pos_target, torch.ones(pos_target.shape[0])), max_iter=140, l=9e-4, lr=5e-2, log_interval=1)

It: 0, deformation cost: 0.000000, attach: 26.811464. Total cost: 26.811464


It: 1, deformation cost: 28.759995, attach: 95.446487. Total cost: 124.206482


It: 2, deformation cost: 23.678719, attach: 88.697556. Total cost: 112.376274


It: 3, deformation cost: 21.220154, attach: 85.055466. Total cost: 106.275620


It: 4, deformation cost: 11.638828, attach: 61.290871. Total cost: 72.929703


...

It: 136, deformation cost: 2.425546, attach: 0.793784. Total cost: 3.219331


It: 137, deformation cost: 2.428182, attach: 0.784575. Total cost: 3.212758


It: 138, deformation cost: 2.429798, attach: 0.776998. Total cost: 3.206797


It: 139, deformation cost: 2.431546, attach: 0.767722. Total cost: 3.199268


It: 140, deformation cost: 2.432353, attach: 0.759647. Total cost: 3.192000
End of the optimisation process.


In [49]:
# Results
%matplotlib qt5
plt.subplot(1, 3, 1)
plt.axis(aabb.get_list())
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')


plt.subplot(1, 3, 2)
plt.axis(aabb.get_list())
out = model.shot_manifold[0].gd.view(-1, 2).detach().numpy()
shot_implicit0 = model.shot_manifold[1].gd.view(-1, 2).detach().numpy()
shot_implicit00 = model.shot_manifold[2].gd.view(-1, 2).detach().numpy()
shot_implicit1 = model.shot_manifold[3].gd[0].view(-1, 2).detach().numpy()
plt.plot(out[:, 0], out[:, 1], '-')
plt.plot(shot_implicit0[:, 0], shot_implicit0[:, 1], 'x')
plt.plot(shot_implicit00[:, 0], shot_implicit00[:, 1], 'o')
plt.plot(shot_implicit1[:, 0], shot_implicit1[:, 1], '.')

plt.subplot(1, 3, 3)
plt.axis(aabb.get_list())
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')
plt.plot(out[:, 0], out[:, 1], '-')
plt.show()

In [ ]:
compound = dm.deformationmodules.CompoundModule(model.modules)
compound.manifold.fill(model.init_manifold.copy())
#compound.manifold.fill_cotan(model.shot_manifold.cotan)

itt = 4
out = dm.shooting.shoot(dm.hamiltonian.Hamiltonian(compound), it=itt)

for i in range(itt):
    plt.figure()
    plt.plot(out[i].gd[0].view(-1, 2)[:, 0].detach().numpy(), out[i].gd[0].view(-1, 2)[:, 1].detach().numpy(), '-')
    plt.plot(out[i].gd[1].view(-1, 2)[:, 0].detach().numpy(), out[i].gd[1].view(-1, 2)[:, 1].detach().numpy(), '.')
    plt.plot(out[i][2].gd.view(-1, 2)[:, 0].detach().numpy(), out[i][2].gd.view(-1, 2)[:, 1].detach().numpy(), 'o')
    plt.plot(out[i][3].gd[0].view(-1, 2)[:, 0].detach().numpy(), out[i][3].gd[0].view(-1, 2)[:, 1].detach().numpy(), 'x')
    plt.show()




In [ ]:
# Evolution of the cost with iterations
plt.title("Cost")
plt.xlabel("Iteration(s)")
plt.ylabel("Cost")
plt.plot(range(len(costs)), costs)
plt.show()